This project shows the comparison of Synthetic Lab tests and medication data generated by 3 LLMs GPT-4.1-mini, Gemini-1.5-pro. and Gemma3:270m. 
The user selects Lab Test Samples, LLM generates synthetic values, and predicts disease and medications. The app shows comparisons.

In [1]:
#imports
import json
import pandas as pd
import gradio as gr
from openai import OpenAI
from dotenv import load_dotenv
import os

In [ ]:
# Load environment variables in a file called .env
# Print the key prefixes to help with any debugging
# You can choose whichever providers you like - or all Ollama

load_dotenv(override=True)
openai_api_key = os.getenv('OPENAI_API_KEY')
google_api_key = os.getenv('GOOGLE_API_KEY')

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
if google_api_key:
    print(f"Google API Key exists and begins {google_api_key[:8]}")
else:
    print("Google API Key not set")

In [3]:
# Connect to OpenAI, gemini and ollama

openai = OpenAI()
gemini_url = "https://generativelanguage.googleapis.com/v1beta/openai/"
gemini = OpenAI(api_key=google_api_key, base_url=gemini_url)
OLLAMA_BASE_URL = "http://localhost:11434/v1"

ollama = OpenAI(base_url=OLLAMA_BASE_URL, api_key='ollama')

In [4]:
#lab tests panel
LAB_PANELS = {
    "Comprehensive Metabolic Panel": [
        "Glucose (mg/dL)",
        "Creatinine (mg/dL)",
        "BUN (mg/dL)",
        "Sodium (mEq/L)",
        "Potassium (mEq/L)"
    ],
    "Complete Blood Count (CBC)": [
        "Hemoglobin (g/dL)",
        "WBC (10^9/L)",
        "Platelets (10^9/L)",
        "MCV (fL)"
    ],
    "Lipid Panel": [
        "Total Cholesterol (mg/dL)",
        "LDL (mg/dL)",
        "HDL (mg/dL)",
        "Triglycerides (mg/dL)"
    ],
    "Thyroid Panel": [
        "TSH (mIU/L)",
        "T3 (ng/dL)",
        "T4 (µg/dL)"
    ]
}

In [13]:
STRUCTURED_PROMPT = """
You are a clinical AI.

Given the following lab test panel:
{panel}

1) Generate realistic abnormal lab values.
2) Predict the most likely disease.
3) Recommend treatment.
4) Provide severity and confidence.

Respond ONLY in JSON format:

{{
  "synthetic_lab_values": {{}},
  "predicted_disease": "",
  "severity": "",
  "recommended_medications": [],
  "confidence_score": 0
}}
"""

In [6]:
def safe_parse(response_text):
    try:
        return json.loads(response_text)
    except:
        return {
            "synthetic_lab_values": {},
            "predicted_disease": "Parsing Failed",
            "severity": "Unknown",
            "recommended_medications": [],
            "confidence_score": 0
        }

In [7]:
def call_gpt(panel):
    prompt = STRUCTURED_PROMPT.format(panel=panel)

    response = openai.chat.completions.create(
        model="gpt-4.1-mini",
        messages=[{"role": "user", "content": prompt}],
        temperature=0.3
    )

    return safe_parse(response.choices[0].message.content)

In [15]:
def call_gemini(panel):
    prompt = STRUCTURED_PROMPT.format(panel=panel)
    response = gemini.chat.completions.create(
        model="gemini-2.5-pro",
        messages=[{"role": "user", "content": prompt}],
        temperature=0.3
    )

    return safe_parse(response.choices[0].message.content)

In [9]:
def call_gemma(panel):
    prompt = STRUCTURED_PROMPT.format(panel=panel)

    response = ollama.chat.completions.create(
        model="gemma3:270m",
        messages=[{"role": "user", "content": prompt}],
        temperature=0.3,
        stream=False
    )

    return safe_parse(response.choices[0].message.content)

In [10]:
#grade llms
def score_llm(response_json):
    score = 0

    if response_json["predicted_disease"] != "Parsing Failed":
        score += 30

    if len(response_json["synthetic_lab_values"]) > 0:
        score += 25

    if len(response_json["recommended_medications"]) > 0:
        score += 20

    if 50 <= response_json["confidence_score"] <= 100:
        score += 25

    return score

In [17]:
#compare llms
def compare_llms(selected_panel):

    panel_tests = LAB_PANELS[selected_panel]

    gpt = call_gpt(panel_tests)
    gemini = call_gemini(panel_tests)
    gemma = call_gemma(panel_tests)

    gpt_score = score_llm(gpt)
    gemini_score = score_llm(gemini)
    gemma_score = score_llm(gemma)

    comparison_table = pd.DataFrame([
        {
            "Model": "GPT-4.1-mini",
            "Predicted Disease": gpt["predicted_disease"],
            "Severity": gpt["severity"],
            "Medications": ", ".join(gpt["recommended_medications"]),
            "Confidence": gpt["confidence_score"],
            "Score": gpt_score
        },
        {
            "Model": "Gemini-2.5-pro",
            "Predicted Disease": gemini["predicted_disease"],
            "Severity": gemini["severity"],
            "Medications": ", ".join(gemini["recommended_medications"]),
            "Confidence": gemini["confidence_score"],
            "Score": gemini_score
        },
        {
            "Model": "Gemma3-270m",
            "Predicted Disease": gemma["predicted_disease"],
            "Severity": gemma["severity"],
            "Medications": ", ".join(gemma["recommended_medications"]),
            "Confidence": gemma["confidence_score"],
            "Score": gemma_score
        }
    ])

    return comparison_table

In [ ]:
#gradio ui
def run_app(selected_panel):
    return compare_llms(selected_panel)

with gr.Blocks(title="AI Doctors Compare") as app:
    gr.Markdown("# 🏥 AI Doctors Compare")
    gr.Markdown("LLMs Generate Synthetic Labs and Predict Disease")

    panel_dropdown = gr.Dropdown(
        choices=list(LAB_PANELS.keys()),
        label="Select Lab Test Panel",
        value="Comprehensive Metabolic Panel"
    )

    comparison_output = gr.Dataframe(label="LLM Comparison Table")

    submit_btn = gr.Button("Run Analysis")
    submit_btn.click(
        run_app,
        inputs=panel_dropdown,
        outputs=comparison_output
    )

app.launch()